In [ ]:
import pydsm
from pydsm import postpro

import pydelmod
from pydelmod import calibplot

VERBOSE=False

In [ ]:
locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_ec_stations.csv'
dfloc = postpro.load_location_file(locationfile)
locations=[postpro.Location(r['Name'],r['BPart'],r['Description']) for i,r in dfloc.iterrows()]
           

In [ ]:
vartype=postpro.VarType('EC','mmhos/cm')
obs_study=postpro.Study('Observed','e:/dsm2v82_calibration/ObservedData/updated_sep-oct2020/cdec_ec_merged_sep2020.dss')

In [ ]:
base_dir='e:/dsm2v82_calibration/'
model_output_dir='output/'
# No "." allowed in study names because of issue https://github.com/holoviz/holoviews/issues/4714
study_file_map = {'DCD': base_dir+'BaselineRun/DSM2_DCD/'+model_output_dir+'historical_v81_DCD.dss',
                  'DCD12 SMCD': base_dir+'BaselineRun/DSM2_DCD1.2_SMCD/'+model_output_dir+'historical_v82b1.dss',
                  'DCD12 SMCD_last3': base_dir+'CalibrationRuns/last/DSM2_DCD1.2_SMCD_last3bin/'+model_output_dir+'historical_v82b1.dss',
                  'DICU': base_dir+'BaselineRun/DSM2_DICU/'+model_output_dir+'historical_v81_DICU.dss'}
model_studies=[postpro.Study(name,study_file_map[name]) for name in study_file_map]

In [ ]:
studies=[obs_study]+model_studies

In [ ]:
location=locations[14] # param
if VERBOSE:
    print(location)

In [ ]:
pp=[postpro.PostProcessor(study,location,vartype) for study in studies]
for p in pp: p.load_processed(timewindow='01OCT2013 - 01DEC2013')

In [ ]:
gridstyle={'grid_line_alpha':1,'grid_line_color':'lightgrey'}

In [ ]:
tsp=calibplot.tsplot([p.df for p in pp],[p.study.name for p in pp]).opts(ylabel=f'{vartype.name} @ {location.name}',show_grid=True,gridstyle=gridstyle)

In [ ]:
gtsp=calibplot.tsplot([p.gdf for p in pp],[p.study.name for p in pp]).opts(ylabel=f'{vartype.name} @ {location.name}',show_grid=True,gridstyle=gridstyle)

In [ ]:
tsplots2=(tsp.opts(width=900)+gtsp.opts(show_legend=False,width=900)).cols(1)
if VERBOSE: 
    tsplots2

In [ ]:
import holoviews as hv
from holoviews import opts
splot=calibplot.scatterplot([p.gdf for p in pp], [p.study.name for p in pp])\
    .opts(opts.Scatter(color=calibplot.shift_cycle(hv.Cycle('Category10'))))\
    .opts(ylabel='Model',legend_position="top_left")\
    .opts(show_grid=True, frame_height=250, frame_width=250, data_aspect=1)
if VERBOSE:
    splot

In [ ]:
dfmetrics=calibplot.calculate_metrics([p.gdf for p in pp],[p.study.name for p in pp])
if VERBOSE:
    dfmetrics

In [ ]:
dfmetrics_monthly=calibplot.calculate_metrics([p.gdf.resample('M').mean() for p in pp],[p.study.name for p in pp])
if VERBOSE:
    dfmetrics_monthly

In [ ]:
slope_plots=calibplot.regression_line_plots(dfmetrics)
cplot=slope_plots.opts(opts.Slope(color=calibplot.shift_cycle(hv.Cycle('Category10'))))*splot
cplot=cplot.opts(xlabel='Observed',ylabel='Model',legend_position="top_left")\
    .opts(show_grid=True, frame_height=250, frame_width=250, data_aspect=1, show_legend=False)
if VERBOSE:
    cplot

In [ ]:
import panel as pn
import pandas as pd

In [ ]:
dfdisplayed_metrics=dfmetrics.loc[:,['regression_equation','r2','mean_error','rmse']]
dfdisplayed_metrics=pd.concat([dfdisplayed_metrics,dfmetrics_monthly.loc[:,['mean_error','rmse']]],axis=1)

In [ ]:
dfdisplayed_metrics.index.name='DSM2 Run'
dfdisplayed_metrics.columns=['Equation','R Squared','Mean Error','RMSE','Monthly Mean Error','Monthly RMSE']
metrics_panel=pn.widgets.DataFrame(dfdisplayed_metrics)
if VERBOSE: 
    metrics_panel

In [ ]:
for p in pp[1:]:
    p.process_diff(pp[0])

In [ ]:
# Needed because of name with . https://github.com/holoviz/holoviews/issues/4714
def sanitize_name(name):
    return name.replace('.',' ')

In [ ]:
amp_diff_kde=calibplot.kdeplot([p.amp_diff for p in pp[1:]],[p.study.name for p in pp[1:]],'Amplitude Diff')
amp_diff_kde=amp_diff_kde.opts(opts.Distribution(color=calibplot.shift_cycle(hv.Cycle('Category10'))))
if VERBOSE:
    amp_diff_kde

In [ ]:
amp_pdiff_kde=calibplot.kdeplot([p.amp_diff_pct for p in pp[1:]],[p.study.name for p in pp[1:]],'Amplitude Diff (%)')
amp_pdiff_kde=amp_pdiff_kde.opts(opts.Distribution(color=calibplot.shift_cycle(hv.Cycle('Category10'))))
if VERBOSE:
    amp_pdiff_kde

In [ ]:
phase_diff_kde=calibplot.kdeplot([p.phase_diff for p in pp[1:]],[p.study.name for p in pp[1:]],'Phase Diff (minutes)')
phase_diff_kde=phase_diff_kde.opts(opts.Distribution(line_color=calibplot.shift_cycle(hv.Cycle('Category10')), filled=True))
if VERBOSE:
    phase_diff_kde

In [ ]:
kdeplots=amp_diff_kde.opts(show_legend=False)+amp_pdiff_kde.opts(show_legend=False)+phase_diff_kde.opts(show_legend=False)
kdeplots=kdeplots.cols(3).opts(shared_axes=False).opts(opts.Distribution(height=200,width=300))
if VERBOSE:
    kdeplots

In [ ]:
header_panel=pn.panel(f'## {location.description} ({location.name}/{vartype.name})')

In [ ]:
calib_plot_template=pn.Column(
    header_panel,
    pn.Row(tsplots2),
    pn.Row(cplot.opts(shared_axes=False, toolbar=None),metrics_panel),pn.Row(kdeplots.opts(toolbar=None)))
if VERBOSE: calib_plot_template

In [ ]:
calib_plot_template_ec=pn.Column(
    header_panel,
    pn.Row(gtsp.opts(width=900, show_legend=True)),
    pn.Row(cplot.opts(shared_axes=False),metrics_panel))

if VERBOSE: calib_plot_template_ec

In [ ]:
calib_plot_template.save('rsan018_flow_stage_template.html')
calib_plot_template_ec.save('rsan018_ec_template.html')